# **Recolección de datos climaticos y de calidad del aire:**
***Open-Meteo Historical Weather API* & *Open-Meteo Air Quality API***

Variables que se incluyeron:

**Variables climáticas**
1. **`temperature_2m_max`**: Temperatura máxima a 2 metros sobre el nivel del suelo (°C).  
2. **`precipitation_sum`**: Precipitación total acumulada durante el día (mm).  
3. **`windgusts_10m_max`**: Velocidad máxima de las ráfagas de viento a 10 metros de altura (m/s).  

**Variables de calidad del aire**
1. **`pm10`**: Concentración horaria de partículas inhalables con diámetro ≤ 10 µm (µg/m³).  
2. **`pm2_5`**: Concentración horaria de partículas finas con diámetro ≤ 2.5 µm (µg/m³).  
3. **`ozone`**: Concentración horaria de ozono troposférico (O₃) (µg/m³).  
4. **`carbon_monoxide`**: Concentración horaria de monóxido de carbono (CO) (mg/m³).  


Open-Meteo Historical Weather API https://open-meteo.com/en/docs/historical-weather-api

Open-Meteo Air Quality API https://open-meteo.com/en/docs/air-quality-api?hourly=pm10,pm2_5,carbon_dioxide,ozone

## **Librerias**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import requests
import geopandas as gpd
from unidecode import unidecode
from funciones.rename_nivel import *
import os

## **Limpieza de datos: Centros poblados**

In [3]:
centro_atlantico = pd.read_csv('data_centros_poblados/raw/centros_atlanticos.csv')
centro_magdalena = pd.read_csv('data_centros_poblados/raw/centros_magdalena.csv')
centro_cesar= pd.read_csv('data_centros_poblados/raw/centros_cesar.csv')
centro_guajira = pd.read_csv('data_centros_poblados/raw/centros_laguajira.csv')
centro_sucre = pd.read_csv('data_centros_poblados/raw/centros_sucre.csv')
cento_sai = pd.read_csv('data_centros_poblados/raw/centros_sanandres.csv')
centro_cordoba = pd.read_csv('data_centros_poblados/raw/centros_cordoba.csv')
centro_bolivar = pd.read_csv('data_centros_poblados/raw/centros_bolivar.csv')

centros =[centro_atlantico, centro_magdalena, centro_cesar, centro_guajira, centro_sucre, cento_sai, centro_cordoba, centro_bolivar]

var=['Código Departamento', 'Código Municipio', 'Código Centro Poblado','Nombre Departamento', 'Nombre Municipio', 'Nombre Centro Poblado','Tipo Centro Poblado', 'Longitud', 'Latitud']

In [4]:
for ce in centros:
    print(ce.shape)

(85, 12)
(312, 12)
(229, 12)
(189, 12)
(378, 12)
(11, 12)
(588, 12)
(490, 12)


In [5]:
total_filas = sum(centro.shape[0] for centro in centros)
print(total_filas)

2282


In [6]:
centros_dict = {}

for centro in centros:
    # Filtrar columnas deseadas
    centro = centro[var]

    # Limpiar nombres
    centro['Nombre Departamento'] = centro['Nombre Departamento'].apply(unidecode)
    centro['Nombre Municipio'] = centro['Nombre Municipio'].apply(unidecode)
    centro['Nombre Centro Poblado'] = centro['Nombre Centro Poblado'].apply(unidecode)

    centro.rename(columns={ 'Código Departamento' : 'COD_DEPTO',
                        'Código Municipio' : 'COD_MUNI',
                        'Código Centro Poblado' : 'COD_CE_POBLADO', 
                        'Nombre Departamento' : 'DEPARTAMENTO',
                        'Nombre Municipio' : 'MUNICIPIO',
                        'Nombre Centro Poblado' : 'CENTRO_POBLADO',
                        'Tipo Centro Poblado' : 'TIPO_CE_POBLADO',
                        'Longitud' : 'LONGITUD',
                        'Latitud' : 'LATITUD'
                       }, inplace=True)
    
    centro.replace({'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA': 'SAN ANDRES'}, inplace=True)

    nombre_dpto = unidecode(centro['DEPARTAMENTO'].iloc[0]).strip().replace(" ", "_").lower()

    # Guardar en el diccionario
    centros_dict[nombre_dpto] = centro
    centro.to_csv(f'data_centros_poblados/clean/centros_{nombre_dpto}_clean.csv', index=False)

In [20]:
centros_dict['cesar']

,COD_DEPTO,COD_MUNI,COD_CE_POBLADO,DEPARTAMENTO,MUNICIPIO,CENTRO_POBLADO,TIPO_CE_POBLADO,LONGITUD,LATITUD
0,20,20001,20001000,CESAR,VALLEDUPAR,VALLEDUPAR,CABECERA MUNICIPAL,-73.259398,10.460472
1,20,20001,20001001,CESAR,VALLEDUPAR,AGUAS BLANCAS,CENTRO POBLADO,-73.487157,10.225763
2,20,20001,20001002,CESAR,VALLEDUPAR,ATANQUEZ,CENTRO POBLADO,-73.353992,10.696908
3,20,20001,20001003,CESAR,VALLEDUPAR,BADILLO,CENTRO POBLADO,-73.128231,10.630880
4,20,20001,20001005,CESAR,VALLEDUPAR,CARACOLI,CENTRO POBLADO,-73.736000,10.078711
...,...,...,...,...,...,...,...,...,...
224,20,20787,20787014,CESAR,TAMALAMEQUE,MUNDO ALREVEZ,CENTRO POBLADO,-73.736723,8.779009
225,20,20787,20787015,CESAR,TAMALAMEQUE,EL DOCE,CENTRO POBLADO,-73.859862,8.891108
226,20,20787,20787016,CESAR,TAMALAMEQUE,SITIO NUEVO,CENTRO POBLADO,-73.756935,8.810860
227,20,20787,20787017,CESAR,TAMALAMEQUE,TOTUMITO,CENTRO POBLADO,-73.738152,8.931471


## **Limpieza para la union de los dataframes**

In [8]:
# Si también quieres el total final:
df_resultado = pd.concat([pd.read_csv(f'data_processed/climate/{archivo}') for archivo in os.listdir('data_processed/climate')])

In [9]:
folder = 'data_processed/climate'

# Opción recomendada: guardarlos en un diccionario
dataframes = {}

for archivo in os.listdir(folder):
    nombre = os.path.splitext(archivo)[0]
    df = pd.read_csv(os.path.join(folder, archivo))
    dataframes[nombre] = df


In [10]:
for nombre, df in dataframes.items():
    print(f"{nombre}: {df.shape}")

data_climate_atlantico: (85, 12)
data_climate_cesar: (225, 12)
data_climate_la_guajira: (160, 12)
data_climate_magdalena: (308, 12)
data_climate_san_andres: (11, 12)
data_climate_sucre: (374, 12)


In [11]:
for nombre, df in centros_dict.items():
    print(f"{nombre}: {df.shape}")

atlantico: (85, 9)
magdalena: (312, 9)
cesar: (229, 9)
la_guajira: (189, 9)
sucre: (378, 9)
san_andres: (11, 9)
cordoba: (588, 9)
bolivar: (490, 9)


In [12]:
# 1) Normalizamos las claves de los CSV procesados quitando el prefijo:
proc = {
    nombre.replace('data_climate_', ''): df
    for nombre, df in dataframes.items()
}

# 2) Usamos directamente tu dict original de centros:
orig = centros_dict

# 3) Encontramos los comunes y los que faltan en cada uno
comunes = set(proc) & set(orig)
solo_proc = set(proc) - set(orig)
solo_orig = set(orig) - set(proc)

print("Departamentos presentes en ambos:", sorted(comunes))
print("Sólo en procesados:        ", sorted(solo_proc))
print("Sólo en originales:        ", sorted(solo_orig), "\n")

Departamentos presentes en ambos: ['atlantico', 'cesar', 'la_guajira', 'magdalena', 'san_andres', 'sucre']
Sólo en procesados:         []
Sólo en originales:         ['bolivar', 'cordoba'] 



In [13]:
for depto in sorted(comunes):
    cod_orig = set(orig[depto]['COD_CE_POBLADO'])
    cod_proc = set(proc[depto]['COD_CE_POBLADO'])
    faltantes = cod_orig - cod_proc
    if faltantes:
        print(f"\n {depto} → faltan {len(faltantes)} códigos:")
        print(sorted(faltantes))



 cesar → faltan 4 códigos:
[20001052, 20001053, 20001054, 20570006]

 la_guajira → faltan 29 códigos:
[44001045, 44001046, 44001047, 44078017, 44078018, 44078019, 44378013, 44430021, 44430022, 44430023, 44430024, 44430025, 44430026, 44430027, 44430028, 44430029, 44560014, 44560015, 44560016, 44560017, 44847039, 44847040, 44847041, 44847042, 44847043, 44847044, 44847045, 44847046, 44847047]

 magdalena → faltan 4 códigos:
[47001046, 47189027, 47189028, 47660016]

 sucre → faltan 4 códigos:
[70001029, 70523019, 70523021, 70670025]


In [50]:
merged = {}

for depto in sorted(comunes):
    df_o = orig[depto]
    df_p = proc[depto]
    nuevas = [c for c in df_p.columns if c not in df_o.columns]

    df_m = df_o.merge(
        df_p[['COD_CE_POBLADO'] + nuevas],
        on='COD_CE_POBLADO',
        how='left'
    )

    merged[depto] = df_m


In [49]:
merged['cesar']

,COD_DEPTO,COD_MUNI,COD_CE_POBLADO,DEPARTAMENTO,MUNICIPIO,CENTRO_POBLADO,TIPO_CE_POBLADO,LONGITUD,LATITUD,temp_max_2023,precipitacion_2023,viento_max_2023
0,20,20001,20001000,CESAR,VALLEDUPAR,VALLEDUPAR,CABECERA MUNICIPAL,-73.259398,10.460472,33.894795,2.719452,7.877808
1,20,20001,20001001,CESAR,VALLEDUPAR,AGUAS BLANCAS,CENTRO POBLADO,-73.487157,10.225763,35.716986,2.856438,8.415890
2,20,20001,20001002,CESAR,VALLEDUPAR,ATANQUEZ,CENTRO POBLADO,-73.353992,10.696908,27.853699,8.373425,7.640822
3,20,20001,20001003,CESAR,VALLEDUPAR,BADILLO,CENTRO POBLADO,-73.128231,10.630880,33.340000,4.310137,8.578630
4,20,20001,20001005,CESAR,VALLEDUPAR,CARACOLI,CENTRO POBLADO,-73.736000,10.078711,36.353973,3.185479,9.176712
...,...,...,...,...,...,...,...,...,...,...,...,...
224,20,20787,20787014,CESAR,TAMALAMEQUE,MUNDO ALREVEZ,CENTRO POBLADO,-73.736723,8.779009,32.638356,10.060000,7.943836
225,20,20787,20787015,CESAR,TAMALAMEQUE,EL DOCE,CENTRO POBLADO,-73.859862,8.891108,32.646575,9.301096,7.590685
226,20,20787,20787016,CESAR,TAMALAMEQUE,SITIO NUEVO,CENTRO POBLADO,-73.756935,8.810860,32.738356,10.060000,7.943836
227,20,20787,20787017,CESAR,TAMALAMEQUE,TOTUMITO,CENTRO POBLADO,-73.738152,8.931471,32.546575,9.301096,7.590685
